In [2]:
%reload_ext autoreload
%autoreload 2
import slap2_py as spy

#import wisco_slap as wis

#import wisco_slap.defs as DEFS
#import xarray as xr
#import matplotlib.pyplot as plt
#import pandas as pd
#import os
#import numpy as np
#import matplotlib as mpl
#import h5py

In [2]:
p = wis.utils.sub_esum_path('avior', 'exp_1', 'loc_I', 'acq_1')
d = spy.SlapData.from_mat73(p)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)


SlapData.data['E'][TRIAL][DMD]

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(50, 30))
ax[0].imshow(d.mean_im(1, 2), vmin=0, vmax=30)
ax[0].imshow(
    d.maxfp(1, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0,
)

ax[1].imshow(d.mean_im(2, 2), vmin=0, vmax=30)
ax[1].imshow(
    d.maxfp(2, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0,
)

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(50, 30))
ax[0].imshow(d.mean_im(1, 2), vmin=0, vmax=30)
ax[0].imshow(
    d.maxfp(1, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0.6,
)

ax[1].imshow(d.mean_im(2, 2), vmin=0, vmax=30)
ax[1].imshow(
    d.maxfp(2, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0.6,
)

In [6]:
p = wis.utils.sub_esum_path('avior', 'exp_1', 'loc_z', 'acq_1')
d = spy.SlapData.from_mat73(p)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)


SlapData.data['E'][TRIAL][DMD]

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(35, 20))
ax[0].imshow(d.mean_im(1, 2), vmin=0, vmax=30)
ax[0].imshow(
    d.maxfp(1, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0.5,
)

ax[1].imshow(d.mean_im(2, 2), vmin=0, vmax=30)
ax[1].imshow(
    d.maxfp(2, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0.5,
)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(35, 20))
ax.imshow(d.mean_im(2, 2), vmin=0, vmax=30)
ax.imshow(
    d.maxfp(2, 0),
    vmin=0.1,
    vmax=1.2,
    cmap=mpl.colors.ListedColormap(["white", "red"]),
    alpha=0.5,
)

In [161]:
mf = extract_trace_group(dat, group='dF', trace='matchFilt', extract_chan=1)

In [162]:
fs = float(dat.data['params']['analyzeHz'])
time = np.arange(mf['DMD1'].shape[1]) / fs
x1 = xr.DataArray(mf['DMD1'], dims=['roi', 'time'], coords={'time': time, 'roi': np.arange(mf['DMD1'].shape[0])})
time = np.arange(mf['DMD2'].shape[1]) / fs
x2 = xr.DataArray(mf['DMD2'], dims=['roi', 'time'], coords={'time': time, 'roi': np.arange(mf['DMD2'].shape[0])})

In [163]:
match = xr.Dataset({'DMD1': x1, 'DMD2': x2})

In [ ]:
match

In [ ]:
for r in range(52):
    f, ax = plt.subplots(1, 1, figsize=(40, 5))
    match['DMD2'].sel(roi=r).plot(ax=ax)


In [22]:
def get_acq_timestamp(subject, exp, loc, acq):
    path = f'{DEFS.data_root}/{subject}/{exp}/{loc}/{acq}/datStartTimes1.txt'
    if not os.path.exists(path):
        raise FileNotFoundError(f'File not found: {path}')
    
    starts  = [] 
    #load each line in the file as a pandas timestamp
    with open(path, 'r') as f:
        for line in f:
            starts.append(pd.to_datetime(line.strip()))
    return starts[0]

In [23]:
st = get_acq_timestamp('atria', 'exp_1', 'loc_1', 'acq_2')

In [ ]:
def extract_traces(dat, group='dF', extract_chan=2):
    n_chunks = len(dat.data['E'])
    trace_types = dat.data['E'][0][0][group].keys()
    full_data = {}
    full_data['DMD1'] = {}
    full_data['DMD2'] = {}
    for dmd in [1, 2]:
        for tt in trace_types:
            full_data[f'DMD{dmd}'][tt] = []
            for chunk in range(n_chunks):
                if dat.data['E'][chunk][dmd-1][group][tt].ndim == 3:
                    full_data[f'DMD{dmd}'][tt].append(dat.data['E'][chunk][dmd-1][group][tt][:, :, extract_chan-1])
                elif dat.data['E'][chunk][dmd-1][group][tt].ndim == 2:
                    full_data[f'DMD{dmd}'][tt].append(dat.data['E'][chunk][dmd-1][group][tt])
                else:
                    raise ValueError('Unexpected number of dimensions')
            full_data[f'DMD{dmd}'][tt] = np.concatenate(full_data[f'DMD{dmd}'][tt], axis=1)
    return full_data


In [ ]:
def extract_trace_group(dat, group='dF', trace='matchFilt', extract_chan=2):
    n_chunks = len(dat.data['E'])
    full_data = {}
    full_data['DMD1'] = []
    full_data['DMD2'] = []
    for dmd in [1, 2]:
        for chunk in range(n_chunks):
            if dat.data['E'][chunk][dmd-1][group][trace].ndim == 3:
                full_data[f'DMD{dmd}'].append(dat.data['E'][chunk][dmd-1][group][trace][:, :, extract_chan-1])
            elif dat.data['E'][chunk][dmd-1][group][trace].ndim == 2:
                full_data[f'DMD{dmd}'].append(dat.data['E'][chunk][dmd-1][group][trace])
            else:
                raise ValueError('Unexpected number of dimensions')
        full_data[f'DMD{dmd}'] = np.concatenate(full_data[f'DMD{dmd}'], axis=1)
    return full_data